## Data Description

This data represents the results of a large product testing study. For each `product_code` you are given a number of product `attributes` (fixed for the code) as well as a number of `measurement` values for each individual product, representing various lab testing methods. Each product is used in a simulated real-world environment experiment, and and absorbs a certain amount of fluid (`loading`) to see whether or not it fails. &nbsp;

Your task is to use the data to predict individual product failures of new codes with their individual lab test results.

## Evaluation

Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.

## Understanding

- Same product_code means they are the same products
- Same product consists of same attributes

## Blueprint

1. Numerize 'attribute_0' and 'attribute_1'
2. Drop the product code(A, B, C, D, E)
3. Apply PCA
4. Split into training and validating data
5. Apply ML models

### Import Packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn import metrics     # error function : metrics.roc_auc_score()

# Show all the columns and rows
pd.set_option("display.max_columns", None)  # columns
# pd.set_option("display.max_rows", None)   # rows

## 1. Data Loading

In [2]:
# Load dataset
data = pd.read_csv('train.csv') # training
te = pd.read_csv('test.csv')    # testing

print(data.shape)
data.head()

(26570, 26)


,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,0,A,80.10,material_7,material_8,9,5,7,8,4,18.040,12.518,15.748,19.292,11.739,20.155,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,1,A,84.89,material_7,material_8,9,5,14,3,3,18.213,11.540,17.717,17.893,12.748,17.889,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,2,A,82.43,material_7,material_8,9,5,12,1,5,18.057,11.652,16.738,18.240,12.718,18.288,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,3,A,101.07,material_7,material_8,9,5,13,2,6,17.295,11.188,18.576,18.339,12.583,19.060,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,4,A,188.06,material_7,material_8,9,5,9,2,8,19.346,12.950,16.990,15.746,11.306,18.093,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


In [3]:
# Store 'id'
id = data.id

# Drop 'id'
data = data.drop(columns=['id'])

print(data.shape)
data.head()

(26570, 25)


,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,A,80.10,material_7,material_8,9,5,7,8,4,18.040,12.518,15.748,19.292,11.739,20.155,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,A,84.89,material_7,material_8,9,5,14,3,3,18.213,11.540,17.717,17.893,12.748,17.889,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,A,82.43,material_7,material_8,9,5,12,1,5,18.057,11.652,16.738,18.240,12.718,18.288,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,A,101.07,material_7,material_8,9,5,13,2,6,17.295,11.188,18.576,18.339,12.583,19.060,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,A,188.06,material_7,material_8,9,5,9,2,8,19.346,12.950,16.990,15.746,11.306,18.093,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


## 2. Data Exploration

In [4]:
# Check data types and missing values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26570 entries, 0 to 26569
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_code    26570 non-null  object 
 1   loading         26320 non-null  float64
 2   attribute_0     26570 non-null  object 
 3   attribute_1     26570 non-null  object 
 4   attribute_2     26570 non-null  int64  
 5   attribute_3     26570 non-null  int64  
 6   measurement_0   26570 non-null  int64  
 7   measurement_1   26570 non-null  int64  
 8   measurement_2   26570 non-null  int64  
 9   measurement_3   26189 non-null  float64
 10  measurement_4   26032 non-null  float64
 11  measurement_5   25894 non-null  float64
 12  measurement_6   25774 non-null  float64
 13  measurement_7   25633 non-null  float64
 14  measurement_8   25522 non-null  float64
 15  measurement_9   25343 non-null  float64
 16  measurement_10  25270 non-null  float64
 17  measurement_11  25102 non-null 

In [5]:
# Check the distribution
data["failure"].value_counts()  # target
data["product_code"].value_counts() # product code

C    5765
E    5343
B    5250
D    5112
A    5100
Name: product_code, dtype: int64

### - Treat Missing Values

In [6]:
# Opt A. Drop missing values
data = data.dropna()

# # Opt B. Replace missing values with 0
# data = data.fillna(0)

# # Opt C. Replace missing values with the feature's mean
# data = data.fillna(data.mean())

In [7]:
# Check the distribution again
data["failure"].value_counts()     # target
data["product_code"].value_counts()    # product code

C    2666
B    2420
E    2392
A    2381
D    2324
Name: product_code, dtype: int64

### - Objective Values

In [8]:
# # Unique values in each column
# data['product_code'].unique()   # array(['A', 'B', 'C', 'D', 'E'], dtype=object)
# data['attribute_0'].unique()    # array(['material_7', 'material_5'], dtype=object)
# data['attribute_1'].unique()    # array(['material_8', 'material_5', 'material_6'], dtype=object)

In [9]:
# Check attribute combinations for each product
def combinations(df):
    products = df['product_code'].unique()      # product codes
    attr = []       # list of the combination of attributes

    for product in products:
        attr = []
        subset = df.loc[df['product_code']==product, :]     # get subsets for each 'product code'

        attr.append(subset['attribute_0'].unique())
        attr.append(subset['attribute_1'].unique())
        attr.append(subset['attribute_2'].unique())
        attr.append(subset['attribute_3'].unique())

        print("Product",product, "consists of", attr)

In [10]:
# combinations(data)

In [11]:
# combinations(te)

### - Int/Float Values

In [12]:
data.describe()

,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
count,12183.000000,12183.000000,12183.000000,12183.000000,12183.000000,12183.000000,12183.000000,12183.000000,12183.000000,12183.000000,12183.000000,12183.000000,12183.000000,12183.000000,12183.000000,12183.000000,12183.000000,12183.000000,12183.000000,12183.000000,12183.000000,12183.000000
mean,127.802374,6.764754,7.228515,7.419109,8.236395,6.215546,17.782234,11.734573,17.128798,17.498797,11.715155,19.022710,11.433930,16.137332,19.183826,11.708558,15.638239,16.046096,15.004323,16.464259,700.912242,0.209801
std,39.023844,1.479066,1.457148,4.142730,4.224814,3.303928,1.002298,0.994764,1.002633,1.000499,0.998095,1.002018,0.998223,1.396516,1.523551,1.496890,1.169330,1.488628,1.558413,1.710481,123.456724,0.407183
min,40.810000,5.000000,5.000000,0.000000,0.000000,0.000000,14.100000,8.097000,12.073000,12.715000,7.973000,15.268000,7.537000,9.676000,12.461000,5.167000,11.035000,10.318000,9.158000,10.064000,196.787000,0.000000
25%,100.070000,6.000000,6.000000,4.000000,5.000000,4.000000,17.113000,11.061000,16.433000,16.818000,11.050000,18.338500,10.757500,15.227000,18.163500,10.698000,14.866000,15.055000,13.950500,15.271000,618.136500,0.000000
50%,122.180000,6.000000,8.000000,7.000000,8.000000,6.000000,17.776000,11.733000,17.128000,17.503000,11.712000,19.014000,11.434000,16.158000,19.233000,11.727000,15.624000,16.032000,14.985000,16.443000,700.159000,0.000000
75%,149.190000,8.000000,8.000000,10.000000,11.000000,8.000000,18.469000,12.413000,17.812000,18.169000,12.395000,19.706000,12.101500,17.033500,20.230000,12.732000,16.375500,17.084500,16.038000,17.610000,782.908000,0.000000
max,385.860000,9.000000,9.000000,26.000000,27.000000,24.000000,21.248000,16.484000,21.425000,21.076000,15.243000,23.328000,15.045000,21.459000,25.429000,17.318000,22.388000,22.303000,20.644000,23.164000,1181.998000,1.000000


## 3. Preprocessing

### (1) Numerize 'attribute_0' and 'attribute_1'

In [13]:
# Apply function element-wise
numerized_attr01 = data.loc[:, ['attribute_0','attribute_1']].applymap(lambda x: int(x[9]))

In [14]:
# Drop 'attribute_0', 'attribute_1' and concat
data = data.drop(columns=['attribute_0', 'attribute_1'])
data = pd.concat([numerized_attr01, data], axis=1)

In [15]:
print(data.shape)
data.head()     # 12183(A), 26570(B)

(12183, 25)


,attribute_0,attribute_1,product_code,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
1,7,8,A,84.89,9,5,14,3,3,18.213,11.540,17.717,17.893,12.748,17.889,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
3,7,8,A,101.07,9,5,13,2,6,17.295,11.188,18.576,18.339,12.583,19.060,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,7,8,A,188.06,9,5,9,2,8,19.346,12.950,16.990,15.746,11.306,18.093,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0
7,7,8,A,177.92,9,5,4,8,8,17.062,13.634,17.879,15.894,11.029,18.643,10.254,16.449,20.478,12.207,15.624,16.968,15.176,17.231,684.000,1
11,7,8,A,175.38,9,5,7,3,2,17.029,11.507,18.377,16.338,10.019,20.242,11.309,16.310,18.959,11.520,14.659,15.355,15.175,15.829,792.591,1


In [16]:
# Double check the combination
combinations(data)

Product A consists of [array([7]), array([8]), array([9]), array([5])]
Product B consists of [array([5]), array([5]), array([8]), array([8])]
Product C consists of [array([7]), array([8]), array([5]), array([8])]
Product D consists of [array([7]), array([5]), array([6]), array([6])]
Product E consists of [array([7]), array([6]), array([6]), array([9])]


### (2) Drop 'product_code'(A, B, C, D, E)

In [17]:
# Drop 'product_code'
data = data.drop(columns=['product_code'])

print(data.shape)
data.head()

(12183, 24)


,attribute_0,attribute_1,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
1,7,8,84.89,9,5,14,3,3,18.213,11.540,17.717,17.893,12.748,17.889,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
3,7,8,101.07,9,5,13,2,6,17.295,11.188,18.576,18.339,12.583,19.060,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,7,8,188.06,9,5,9,2,8,19.346,12.950,16.990,15.746,11.306,18.093,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0
7,7,8,177.92,9,5,4,8,8,17.062,13.634,17.879,15.894,11.029,18.643,10.254,16.449,20.478,12.207,15.624,16.968,15.176,17.231,684.000,1
11,7,8,175.38,9,5,7,3,2,17.029,11.507,18.377,16.338,10.019,20.242,11.309,16.310,18.959,11.520,14.659,15.355,15.175,15.829,792.591,1


### (3). Apply PCA

In [18]:
X = data.iloc[:, 0:23]  # all features
y = data.loc[:, 'failure']  # target

In [19]:
# Standardize
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(X)
X_scaled = pd.DataFrame(scaler.transform(X), columns=list(X.columns))

In [20]:
# Check Variance
print(X_scaled.var())

attribute_0       1.000082
attribute_1       1.000082
loading           1.000082
attribute_2       1.000082
attribute_3       1.000082
measurement_0     1.000082
measurement_1     1.000082
measurement_2     1.000082
measurement_3     1.000082
measurement_4     1.000082
measurement_5     1.000082
measurement_6     1.000082
measurement_7     1.000082
measurement_8     1.000082
measurement_9     1.000082
measurement_10    1.000082
measurement_11    1.000082
measurement_12    1.000082
measurement_13    1.000082
measurement_14    1.000082
measurement_15    1.000082
measurement_16    1.000082
measurement_17    1.000082
dtype: float64


In [21]:
# Covariance matrix
# cov = np.dot(X_centered.T, X_centered)/(len(X_centered)-1)
cov = np.dot(X_scaled.T, X_scaled)/(len(X_scaled)-1)
# df_cov = pd.DataFrame(cov)

In [22]:
# Eigenvalues & Eigenvectors
eig = np.linalg.eig(cov)
# print('<eigenvalues>\n', eig[0],'\n')
# print('<eigenvectors>\n', eig[1])

In [23]:
# Contribution in the Data
def percent_variation(n):
    '''Percent Variation of top 'n' factors'''
    trace = sum(eig[0])    # sum of all eigenvalues
    contribution = [np.round(i/trace,5) for i in eig[0]]
    return contribution[:n]    # return the first n number of contributions

In [24]:
n = 22
print(percent_variation(n))
print('Total contribution of 22 components:', sum(percent_variation(n))*100, '%\n')

[0.10158, 0.08263, 0.08029, 0.00553, 0.00673, 0.01987, 0.02841, 0.05289, 0.03174, 0.0343, 0.0351, 0.04966, 0.04739, 0.03865, 0.03941, 0.04089, 0.04528, 0.04433, 0.04396, 0.04231, 0.04334, 0.04302]
Total contribution of 22 components: 95.731 %



In [25]:
# Fit to PCA (getting a projection matrix)
pca = PCA(n_components=22)
pca.fit(X_scaled)

PCA(n_components=22)

In [26]:
# How much each of the features influences the PC
influence = pd.DataFrame(pca.components_, columns=list(X_scaled.columns))
influence

,attribute_0,attribute_1,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,0.213944,0.363880,0.004961,0.370126,-0.518162,0.393531,-0.417895,-0.136903,0.008407,-0.011028,0.018971,-0.001214,0.015466,0.004011,0.014586,0.029100,0.064880,0.154112,-0.041706,0.033156,-0.187061,0.012831,0.015503
1,-0.536593,-0.383341,-0.013957,0.456840,-0.097140,-0.065557,-0.111072,0.284423,-0.032330,-0.058964,-0.117441,-0.097685,-0.086422,-0.147753,-0.053238,0.040750,0.089000,0.244833,0.152435,-0.000331,-0.112234,0.162872,-0.239534
2,-0.207875,-0.154349,0.009813,0.157674,-0.027500,-0.030713,-0.038835,0.115399,0.075259,0.142102,0.349500,0.262361,0.267299,0.376030,0.103972,0.037091,-0.004064,0.065178,0.046291,0.026183,-0.034236,0.048850,0.662539
3,-0.129670,-0.032140,0.047749,0.072533,-0.075849,0.118978,-0.083639,0.114552,-0.021247,-0.075278,-0.007744,0.003526,0.038506,-0.034720,-0.051808,0.197384,-0.562514,-0.323701,0.136358,0.535247,0.102735,-0.382099,-0.034935
4,0.024119,0.117907,-0.095508,-0.044906,-0.015138,0.111443,-0.072776,-0.076843,0.035406,0.109775,-0.092674,-0.038724,-0.024779,0.101754,-0.057793,-0.008205,-0.237595,-0.105859,0.653731,-0.136910,0.338528,0.533251,0.006091
5,0.203579,-0.071660,0.072443,-0.174566,-0.112681,-0.055173,0.280354,0.010377,-0.030317,0.019033,0.051372,-0.091156,-0.108741,0.102187,-0.092283,0.677861,-0.124310,0.254944,0.151136,0.021145,-0.447639,0.128873,-0.004063
6,0.019589,-0.019907,-0.487265,-0.008544,-0.015356,-0.036127,0.037832,0.022377,0.473223,0.463508,-0.233033,-0.124714,0.130421,-0.153051,0.386816,-0.017184,-0.127489,0.043619,-0.008066,0.030383,-0.181922,-0.083309,-0.008548
7,-0.010406,0.044954,0.210080,0.001775,0.041974,-0.036542,-0.023798,-0.054688,0.255621,0.306659,0.607014,0.083478,-0.310107,-0.522739,-0.140763,-0.078950,-0.014976,0.014951,0.112956,0.049799,-0.029334,0.015578,-0.002005
8,0.036859,-0.024971,0.316151,-0.028049,-0.042269,0.023796,0.029268,0.075786,0.366946,-0.448654,-0.158130,0.468579,-0.070008,-0.154912,0.463727,0.032135,-0.067297,-0.078912,0.118056,-0.165103,-0.093245,0.045035,-0.012906
9,-0.025205,-0.017466,-0.090779,0.036763,-0.051299,0.063054,-0.002368,0.096725,-0.198460,0.174722,0.053311,0.015996,-0.751215,0.293826,0.428254,0.083186,0.036741,-0.072718,-0.131741,0.043463,0.183202,-0.006578,0.004305


In [27]:
# Projection (transforming the original data via projection matrix)
data_pca = pca.transform(X_scaled)
data_pca = pd.DataFrame(data_pca)

print(data_pca.shape)
data_pca.head()

(12183, 22)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
0,3.260665,-0.312278,0.040958,0.818371,-0.384500,0.035156,1.227123,0.151427,0.592466,-0.456617,-1.572350,-0.146938,-0.183608,0.357188,0.362714,-1.572102,-0.285381,0.907981,-0.191419,-0.096829,-0.154097,0.342809
1,2.767491,-0.766793,1.544280,0.745061,0.656924,-1.254109,-0.359750,-0.043076,0.508556,0.380710,-1.971382,0.074903,-0.324353,-0.018468,0.567281,-0.540652,0.066448,0.560059,-0.589796,0.341833,-0.167341,0.280553
2,2.789070,1.323082,-1.258936,-1.120785,0.321443,0.865485,0.509401,1.536346,-0.112721,-0.865299,1.597225,1.106557,-0.885807,-0.959948,2.326200,-0.531583,-0.595596,-0.637292,-0.177871,0.354449,-0.313022,-0.165597
3,1.583203,0.743812,-0.319327,-0.305038,0.017965,0.594974,-0.641655,1.564600,-2.162277,0.445272,0.809680,1.130632,0.537839,-1.248084,0.117825,0.353671,-0.160539,-0.537375,-0.655796,1.677118,-0.824910,-0.291324
4,2.512860,-0.541723,0.363994,-0.041118,-0.378988,0.003486,-1.723277,0.511431,-0.814083,1.731384,1.113644,1.070427,-0.740447,0.866798,0.505833,-0.823745,0.043336,0.209304,-1.041912,0.826391,0.117653,-0.433103


## 4. Apply ML models

In [28]:
# Split into training and validating
PCA_train, PCA_val, y_train, y_val = train_test_split(data_pca, y, test_size=0.33, 
                                                    random_state=0, stratify=y)

### (1) Linear SVC

In [29]:
from sklearn.svm import LinearSVC

LinSVC = LinearSVC(random_state=0)
LinSVC.fit(PCA_train, y_train)
y_LinSVC = LinSVC.predict(PCA_val)

/Users/lucy/Library/Python/3.8/lib/python/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [30]:
# Compute the area under the ROC curve
metrics.roc_auc_score(y_val, y_LinSVC)  # 0.5(A), 0.5(B), 0.5(C)

0.5

### (2) SGD Classifier

In [31]:
from sklearn.linear_model import SGDClassifier

SGD = SGDClassifier(random_state=0)
SGD.fit(PCA_train, y_train)
y_SGD = SGD.predict(PCA_val)

In [32]:
# Compute the area under the ROC curve
metrics.roc_auc_score(y_val, y_SGD)     # 0.5(A), 0.5(B), 0.5(C)

0.5

### (3) KNN Classifier

In [33]:
from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier(n_neighbors=2)
KNN.fit(PCA_train, y_train)
y_KNN = KNN.predict(PCA_val)

In [34]:
# Compute the area under the ROC curve
metrics.roc_auc_score(y_val, y_KNN)     # 0.497(A), 0.505(B), 0.503(C)

0.4970903129274839

### (4) Kernel Approximation

In [ ]:
# check

### (5) SVC

In [ ]:
from sklearn.svm import SVC

SVC = SVC(random_state=0)
SVC.fit(PCA_train, y_train)
y_SVC = SVC.predict(PCA_val)

In [ ]:
# Compute the area under the ROC curve
metrics.roc_auc_score(y_val, y_SVC)     # 0.5(A), 0.5(B), 0.5(C)

### (6) Ensemble Classifiers

In [ ]:
from sklearn.ensemble import RandomForestRegressor

Ensb = RandomForestRegressor(max_depth=5, random_state=0)
Ensb.fit(PCA_train, y_train)
y_Ensb = SVC.predict(PCA_val)

In [ ]:
# Compute the area under the ROC curve
metrics.roc_auc_score(y_val, y_Ensb)    # 0.5(A), 0.5(B), 0.5(C)

## 5. Apply The Best-fit Model to The Testing Data